In [ ]:
!pip install -q --upgrade nltk gensim bokeh pandas

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/pi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.manifold import TSNE

/home/pi/.local/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from string import punctuation

## Запилим пословный машинный перевод!

In [ ]:
!wget -O ukr_rus.train.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1vAK0SWXUqei4zTimMvIhH3ufGPsbnC_O"
!wget -O ukr_rus.test.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1W9R2F8OeKHXruo2sicZ6FgBJUTJc8Us_"
!wget -O fairy_tale.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1sq8zSroFeg_afw-60OmY8RATdu_T1tej"

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': '1d7OXuil646jUeDS1JNhP9XWlZogv6rbu'})
downloaded.GetContentFile('cc.ru.300.vec.zip')

downloaded = drive.CreateFile({'id': '1yAqwqgUHtMSfGS99WLGe5unSCyIXfIxi'})
downloaded.GetContentFile('cc.uk.300.vec.zip')

!unzip cc.ru.300.vec.zip
!unzip cc.uk.300.vec.zip

Напишем простенькую реализацию модели машинного перевода.

Идея основана на статье [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087.pdf). У авторов в репозитории еще много интересного: [https://github.com/facebookresearch/MUSE](https://github.com/facebookresearch/MUSE).

А мы будем переводить с украинского на русский.

![](https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/blue_cat_blue_whale.png)   
*синій кіт* vs. *синій кит*

In [4]:
from gensim.models import KeyedVectors

ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

Посмотрим на пару серпень-август (являющихся переводом)

In [5]:
ru_emb.most_similar([ru_emb["август"]])

[('август', 1.0000001192092896),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545795679092407),
 ('февраль', 0.8401415944099426)]

In [6]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999998807907104),
 ('липень', 0.9096441268920898),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992518782615662),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633180618286),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

In [7]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.2757962644100189),
 ('ЖИЗНИВадим', 0.25203436613082886),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829229712486267),
 ('Ведущий-Алексей', 0.24438698589801788),
 ('Недопустимость', 0.24435284733772278),
 ('2Михаил', 0.23981398344039917),
 ('лексей', 0.23740755021572113),
 ('комплексн', 0.23695150017738342),
 ('персональ', 0.2368222326040268)]

In [8]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf8') as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)


uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

### Учим маппинг из одного пространства эмбеддингов в другое

У нас есть пары слов, соответствующих друг другу, и их эмбеддинги. Найдем преобразование из одного пространства в другое, чтобы приблизить известные нам слова:

$$W^*= \arg\min_W ||WX - Y||_F, \text{где} ||*||_F - \text{норма Фробениуса}$$

Эта функция очень похожа на линейную регрессию (без биаса).

**Задание** Реализуйте её - воспользуйтесь `LinearRegression` из sklearn с `fit_intercept=False`:

In [12]:
from sklearn.linear_model import LinearRegression

In [13]:
mapping = LinearRegression(fit_intercept=False).fit(X_train, Y_train)

Проверим, куда перейдет `серпень`:

In [122]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
X_test

array([[ 0.0818, -0.0011,  0.0303, ..., -0.0224, -0.0932, -0.0329],
       [ 0.0796,  0.0098,  0.0155, ...,  0.0401, -0.0148,  0.0391],
       [ 0.1395, -0.0844, -0.04  , ..., -0.0829,  0.0105, -0.019 ],
       ...,
       [ 0.2131,  0.0109, -0.0381, ..., -0.0246,  0.0313,  0.0191],
       [-0.0013,  0.0027,  0.0451, ...,  0.0746,  0.0747,  0.0881],
       [ 0.1414, -0.0753,  0.1002, ...,  0.0549, -0.0555,  0.0288]],
      dtype=float32)

In [14]:
ru_emb.most_similar(august)

[('апрель', 0.8531433343887329),
 ('июнь', 0.8402523398399353),
 ('март', 0.8385884761810303),
 ('сентябрь', 0.8331484794616699),
 ('февраль', 0.8311209082603455),
 ('октябрь', 0.8278019428253174),
 ('ноябрь', 0.8243728876113892),
 ('июль', 0.822961688041687),
 ('август', 0.8112280368804932),
 ('январь', 0.8022985458374023)]

Должно получиться, что в топе содержатся разные месяцы, но август не первый.

Будем мерять percision top-k с k = 1, 5, 10.

**Задание** Реализуйте следующую функцию:

In [134]:
list(august)[0]

array([ 0.00386969, -0.04362756, -0.03990221, -0.0346876 ,  0.00101105,
       -0.03415707, -0.0042239 ,  0.0159358 ,  0.02767768,  0.03029998,
       -0.0067464 ,  0.04458402, -0.01162009,  0.01961648,  0.02015473,
       -0.02718292,  0.05103142,  0.00415257, -0.07797018,  0.0330327 ,
        0.04557055, -0.0543474 , -0.03034731,  0.00096908,  0.04288891,
        0.04933845,  0.00315177, -0.03174236,  0.03080017, -0.00571151,
       -0.02271756,  0.01093677,  0.02068366, -0.08168949,  0.01333102,
       -0.01455903, -0.05045948, -0.16685529, -0.03216281,  0.02048085,
        0.00418746, -0.04094633, -0.00391188, -0.01237782, -0.0147362 ,
        0.00128612,  0.03798787,  0.0850026 ,  0.00266601,  0.05425645,
        0.03481624,  0.01868033,  0.02387061,  0.02912951, -0.01397864,
        0.01304335, -0.02962371, -0.06807907, -0.05696433, -0.02618901,
       -0.07945007,  0.053929  , -0.01163233,  0.05580141,  0.01425152,
       -0.00905488, -0.00679328, -0.01195392, -0.01193032,  0.00

In [140]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        mapped_vector = mapped_vectors[i]
        for j in range(topn):
            if ru_emb.most_similar([mapped_vector])[j][0] == ru:
                num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [141]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [142]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [143]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)
precision_top5

0.7913486005089059

In [145]:
precision_top1

0.628498727735369

In [151]:
#assert precision_top1 >= 0.635
#assert precision_top5 >= 0.813

### Улучшаем маппинг

Можно показать, что маппинг лучше строить ортогональным:
$$W^*= \arg\min_W ||WX - Y||_F \text{, где: } W^TW = I$$

Искать его можно через SVD:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$

$$W^*=UV^T$$

**Задание** Реализуйте эту функцию.

In [146]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    U, D, VT = np.linalg.svd(X_train.T @ Y_train)
    return U@VT

In [147]:
W = learn_transform(X_train, Y_train)

In [148]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8245131969451904),
 ('июнь', 0.8056630492210388),
 ('сентябрь', 0.8055761456489563),
 ('март', 0.8032935261726379),
 ('октябрь', 0.7987101674079895),
 ('июль', 0.7946797013282776),
 ('ноябрь', 0.793963611125946),
 ('август', 0.7938188910484314),
 ('февраль', 0.7923860549926758),
 ('декабрь', 0.7715376019477844)]

In [149]:
precision(uk_ru_test, np.matmul(X_test, W))

0.6437659033078881

In [150]:
precision(uk_ru_test, np.matmul(X_test, W), 5)

0.7989821882951654

In [ ]:
#assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
#assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

### Пишем переводчик

Реализуем простой пословный переводчик - для каждого слова будем искать его ближайшего соседа в общем пространстве эмбеддингов. Если слова нет в эмбеддингах - просто копируем его.

In [154]:
with open("fairy_tale.txt", "r") as f:
    uk_sentences = [line.rstrip().lower() for line in f]

In [167]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    ru_sentence = []
    for word in sentence.split():
        if word not in uk_emb:
            continue
        ru_sentence.append(ru_emb.most_similar([np.matmul(uk_emb[word], W)])[0][0])
    translation = " ".join(ru_sentence)
    return translation

In [168]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [169]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк –

src: як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !
dst: как была себе лисичка , че и пошла раз к обеих бабы добывать огнь ; вошла во избу че и говорит : " хороший день тебе , бабушку !

src: дай мені огня " .
dst: дай мне огня " .

src: а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .
dst: а бабка только что вынула со печи самогончик со маком , сладкий , че и прогнула , чтобы он успокоился ; а лисичка ой и , че токмо что бабка свесилась во печь , чтобы прость огня , то лисичка сейчас самогончик че и дунула со хаты , че , пробежать , весь мак со его кринки , а туду мусор наложила .

src:

KeyboardInterrupt: 

# Дополнительные материалы

## Почитать
### База:  
[On word embeddings - Part 1, Sebastian Ruder](http://ruder.io/word-embeddings-1/)  
[Deep Learning, NLP, and Representations, Christopher Olah](http://colah.github.io/posts/2014-07-NLP-RNNs-Representations/)  

### Как кластеризовать смыслы многозначных слов:  
[Making Sense of Word Embeddings (2016), Pelevina et al](http://anthology.aclweb.org/W16-1620)    

### Как оценивать эмбеддинги
[Evaluation methods for unsupervised word embeddings (2015), T. Schnabel](http://www.aclweb.org/anthology/D15-1036)  
[Intrinsic Evaluation of Word Vectors Fails to Predict Extrinsic Performance (2016), B. Chiu](https://www.aclweb.org/anthology/W/W16/W16-2501.pdf)  
[Problems With Evaluation of Word Embeddings Using Word Similarity Tasks (2016), M. Faruqui](https://arxiv.org/pdf/1605.02276.pdf)  
[Improving Reliability of Word Similarity Evaluation by Redesigning Annotation Task and Performance Measure (2016), Oded Avraham, Yoav Goldberg](https://arxiv.org/pdf/1611.03641.pdf)  
[Evaluating Word Embeddings Using a Representative Suite of Practical Tasks (2016), N. Nayak](https://cs.stanford.edu/~angeli/papers/2016-acl-veceval.pdf)  


## Посмотреть
[Word Vector Representations: word2vec, Lecture 2, cs224n](https://www.youtube.com/watch?v=ERibwqs9p38)